## Download Zip with HTML files.

The zip contain 1 zip each anime with HTML files. With this HTML files we can extract information about the anime like genre, stats, etc. In this case, we will extract the sypnosis of each anime which not include hentai in genres.

In [ ]:
!mkdir -p ../temp/
!mkdir -p ../temp/html

!wget -nc https://raw.githubusercontent.com/Hernan4444/MyScripts/master/google_drive/google_api.py -q --show-progress
!mv google_api.py ../temp/google_api.py

In [ ]:
import sys
sys.path.append('/kaggle/temp/')

import os
from google_api import download_file_without_authenticate

if not os.path.exists('/kaggle/temp/html.zip'): 
    download_file_without_authenticate('1laYyz5WVuaxDhhpLmIvCCFsYRAXvaEoF', '/kaggle/temp/html.zip')
    !unzip -q -o /kaggle/temp/html.zip  -d /kaggle/temp/html 

print(os.listdir("/kaggle/temp/html")[:10])

!ls /kaggle/temp

## Get synopsis for all anime

In [ ]:
import pandas as pd

df = pd.read_csv("../input/anime-recommendation-database-2020/anime.csv")
print(df.shape)

In [ ]:
df_minor = df[df.Genders.apply(lambda x: "Hentai" not in x.split(", "))]
df_minor = df_minor[["MAL_ID", "Name", "Score", "Genders"]]

print(df_minor.shape)
df_minor.head(2)

In [ ]:
all_genders = set()
for i, x in df_minor.iterrows():
    all_genders = all_genders.union(set(x.Genders.split(", ")))

print(all_genders)

In [ ]:
!pip3 install bs4 -q

In [ ]:
from zipfile import ZipFile
from bs4 import BeautifulSoup
import re


def extract_zip(input_zip):
    input_zip = ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

no_synopsis = "No synopsis has been added for this series yet. Click here to update this information."
def get_synopsis_anime(anime_id):
    data = extract_zip(f"/kaggle/temp/html/{anime_id}.zip")
    anime_info = data["details.html"].decode()
    soup = BeautifulSoup(anime_info, "html.parser")
    div = soup.find("div", {"id": "contentWrapper"})
    p = div.find("p", {"itemprop": "description"})
    
    text = re.sub(r"\(Source:\s?..*\)", " ", p.text)
    text = re.sub(r"\[Written by MAL..*]", " ", text)
    text = " ".join(text.split()).strip()
    if text == no_synopsis:
        return "Unknown"
    return text


In [ ]:
for ID in [1115, 1109, 1020, 1023, 2636]:
    print(get_synopsis_anime(ID))
    print()

In [ ]:
%%time

sypnopsis = []
for i, x in enumerate(df_minor.MAL_ID):
    sypnopsis.append(get_synopsis_anime(x))
    if i%500 == 0:
        print(f"\r {i + 1}/{df_minor.shape[0]}", end="")

df_minor["synopsis"] = sypnopsis
df_minor.head()

In [ ]:
df_minor.to_csv("anime_with_synopsis.csv", index=False)